In [ ]:
from datetime import date
import itertools
import logging
import pandas as pd
import numpy as np
import datetime
import glob
import os
import pickle
import ete3
import geopy
import re
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
VALIDATION_VERSION = '0.2.0'
ANOSPP_VERSION = '4.0'
BIOSCAN_VERSION = '3.0'

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)
# logging.getLogger().setFormat('[%(levelname)s] %(message)s')

def setup_logging(verbose=False):
    try: 
        del logging.root.handlers[:]
    except:
        pass
    if verbose:
        logging.basicConfig(level=logging.DEBUG, format='[%(levelname)s] %(message)s')
    else:
        logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')
setup_logging(verbose=True)   
logging.debug('test')

In [ ]:
anospp_fn = '../data/Anopheles_Metadata_Manifest_V4.0_20221220.xlsx'
biosc_fn = '../data/BIOSCAN_Manifest_V2.0_20230727.xlsx'

In [ ]:
# download and install taxonomy
ncbi = ete3.NCBITaxa()
# run update if needed
# ncbi.update_taxonomy_database()

In [ ]:
def get_data(fn, sheet='TAB 2 Metadata Entry'):

    logging.debug(f'reading data from "{fn}" sheet "{sheet}"')
    
    df = pd.read_excel(fn, dtype=str, index_col=0, keep_default_na=False,
                       sheet_name=sheet)
        
    return df

# df = get_data(anospp_fn, sheet='TAB 3 TEST Metadata Entry')

In [ ]:
def fix_date_formats(df):
    # time can be appended to date by conversion to string
    # fixing date formats appearing due to string conversion on reading
    logging.debug('fixing date formats by removing appended times')
    
    for col in ['DATE_OF_COLLECTION', 'DATE_OF_PLATING', 'DATE_OF_PRESERVATION']:
        if col in df.columns:
            df[col] = df[col].str.replace(r' 00:00:00$','')
            
    return df

In [ ]:
def infer_bioscan_version(df):
    
    logging.debug(f'inferring bioscan version from data format')
    
    version_evidence = {}
    
    for col in ('CATCH_SOLUTION', 'AMOUNT_OF_CATCH_PLATED'):
        if col in df.columns:
            version_evidence[col] = 'v3'
        else:
            version_evidence[col] = 'v2'
            
    v3_plate_only = (('PLATE_ONLY_1_BLANK' in df['TUBE_OR_WELL_ID']) 
                     or ('PLATE_ONLY_2_BLANKS' in df['TUBE_OR_WELL_ID']))
    if 'PLATE_ONLY' in df['TUBE_OR_WELL_ID']:
        if v3_plate_only:
            logging.error('found both bioscan v2 and v3 style PLATE_ONLY entries in manifest')
            raise ValueError('fix bioscan manifest version before proceeding')
        else:
            version_evidence['PLATE_ONLY'] = 'v2'
    elif v3_plate_only:
        version_evidence['PLATE_ONLY'] = 'v3'
        
    if len(set(version_evidence.values())) > 1:
        msg = 'found conflicting evidence on bioscan version: '
        for k, v in version_evidence.items():
            msg += f'{k} suggests {v}, '
        logging.error(msg[:-2])
        raise ValueError('fix bioscan manifest version before proceeding')
        
    # returns only if all evidence is concordant
    v = version_evidence['CATCH_SOLUTION']
    logging.info(f'bioscan manifest {v} inferred')
    
    return v

In [ ]:
def validate_series(df):
    
    # series should be 1,2, ..., nsamples
    logging.debug('validating SERIES column')
    
    if df.index.duplicated().any():
        logging.error(f'duplicate SERIES column values: {df.index[df.index.duplicated()].to_list()}, '
                      f'references to these SERIES can be wrong')
    
    # exclude non-numeric SERIES - these don't work with ranges
    series_numeric = df.index.astype(str).str.isnumeric()
    if not series_numeric.all():
        logging.error(f'found rows with non-numeric SERIES column values: '
                      f'{df.index[~series_numeric].to_list()} - '
                      f'these will be excluded from validation and output')
        df = df.loc[series_numeric]
        
    # check the remaining SERIES are continuous
    expected_series = set([str(i) for i in range(1, df.shape[0] + 1)])
    observed_series = set(df.index.astype(str))
    if expected_series != observed_series:
        logging.error(f'SERIES column values {sorted(list(expected_series - observed_series))} are missing, '
                      f'{sorted(list(observed_series - expected_series))} are unexpected')
        
    return df
        
# df = validate_series(df)

In [ ]:
def index_ranges(df):
    
    # based on https://stackoverflow.com/questions/4628333/converting-a-list-of-integers-into-range-in-python
    i = df.index.astype(int).to_list()
    ranges = []
    for a, b in itertools.groupby(enumerate(i), lambda pair: pair[1] - pair[0]):
        b = list(b)
        if b[0][1] != b[-1][1]:
            ranges.append(f'{b[0][1]}-{b[-1][1]}')
        else:
            ranges.append(f'{b[0][1]}')
            
    return ', '.join(ranges)

# index_ranges(df)

In [ ]:
def remove_trailing_spaces(df):
    for col in df.columns:
        trailing_spaces = (df[col].str.startswith(' ') | df[col].str.endswith(' '))
        if trailing_spaces.any():
            logging.info(f'trailing spaces found in {col} column, '
                         f'SERIES {index_ranges(df.loc[trailing_spaces])} - removing for validation and output')
            df[col] = df[col].str.strip()
            
    return df

# df = remove_trailing_spaces(df)

In [ ]:
def remove_nonbreaking_spaces(df):
    for col in df.columns:
        nonbreaking_spaces = df[col].str.contains(u"\u00A0")
        if nonbreaking_spaces.any():
            logging.info(f'non-breaking spaces found in {col} column, '
                         f'SERIES {index_ranges(df.loc[nonbreaking_spaces])} - removing for validation and output')
            df[col] = df[col].str.replace(u"\u00A0", " ")
            
    return df

In [ ]:
def check_columns(df, template_df):
    
    logging.debug('checking manifest columns against template')
    
    data_cols = set(df.columns)
    template_cols = set(template_df.columns)
        
    if data_cols - template_cols != set():
        logging.info(f'extra columns in filled manifest compared to template: {data_cols - template_cols}')
    if template_cols - data_cols != set():
        logging.error(f'template columns missing from filled manifest: {template_cols - data_cols}')

# check_columns(df, template_df)

In [ ]:
def get_valid_dict(fn, validation_sheet='Data Validation - do not edit'):
    
    # pick up validation values from data validation sheet
    logging.debug(f'extracting value validation data from {fn} sheet "{validation_sheet}"')
    valid_df = pd.read_excel(fn, dtype=str, sheet_name=validation_sheet)
    valid_dict = dict()
    for col in valid_df.columns:
        valid_dict[col] = valid_df[col].dropna().to_list()
    
    return valid_dict

# valid_dict = get_valid_dict(anospp_fn, validation_sheet='TAB 5 Data Validation - do not ')

In [ ]:
def exclude_missing(series, na_values=[]):
    
    # valid missing data 
    if len(na_values) > 0:
        no_data = (series.isin(na_values))
        logging.debug(f'excluding {no_data.sum()} samples with missing values ({na_values}) '
                      f'in column {series.name}')
        return series[~no_data]
    return series
    
# exclude_missing(df['TIME_OF_COLLECTION'], na_values=['NOT_COLLECTED',''])

In [ ]:
def validate_contributors(fn, contrib_sheet='TAB 1 Contributors'):
    
    logging.debug(f'validating contributors in {fn}')
        
    df = pd.read_excel(fn, dtype=str, keep_default_na=False,
                       sheet_name=contrib_sheet)
    
    df = remove_trailing_spaces(df)
    
    expected_columns = ['SURNAME','FIRST_NAME','PRIMARY_AFFILIATION','EMAIL ADDRESS','CONTRIBUTION','CONFIRMATION']
    
    if set(df.columns) != set(expected_columns):
        ec = set(df.columns) - set(expected_columns)
        mc = set(expected_columns) - set(df.columns)
        logging.error(f'mismatch in contributor columns: extra {ec}, missing {mc}')
    
    for delim_char in (';','|'):
        for col in df.columns:
            if df[col].str.contains(delim_char, regex=False).any():
                logging.error(f'contributor column {col} contains delimiter "{delim_char}"')
    
    df['FULL_NAME'] = df['FIRST_NAME'] + ' ' + df['SURNAME']
    df['PARTNER_CODE'] = (df['SURNAME'].str.slice(0,2) + df['FIRST_NAME'].str.slice(0,2)).str.upper()
    is_template_name = (df['FULL_NAME'] == 'Charles R. Darwin')
    if is_template_name.any():
        logging.warning('suspect template contributor was not removed')
    
    is_dup_name = df['FULL_NAME'].duplicated()
    if is_dup_name.any():
        logging.error('duplicated contributor names {}'.format(
                df.loc[is_dup_name, 'FULL_NAME'].to_list()))
    
    # TODO update template with underscore in email column
    is_valid_email = df['EMAIL ADDRESS'].str.match('^[A-z0-9._%+-]+@[A-z0-9.-]+\.[A-z]{2,}$')
    if not is_valid_email.all():
        logging.error('invalid contributor email addresses {}'.format(
                df.loc[~is_valid_email, 'EMAIL ADDRESS'].to_list()))
    
    is_confirmed = (df['CONFIRMATION'] == 'YES')
    if not is_confirmed.any():
        logging.error('confirmation lacking for any of contributors')
        
    return df
        
# contrib_df = validate_contributors(anospp_fn)

In [ ]:
def validate_regex(col, df, na_values=[]):
    
    logging.debug(f'validating data format in {col} column')
    
    if col not in df.columns:
        logging.error(f'{col} column not found in manifest')
        return
    series = df[col]
    series = exclude_missing(series, na_values)
    
    # dates between 1900 and 2100
    date_regex = (r'^(19\d\d|20\d\d)-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])$|'
                  r'^(19\d\d|20\d\d)-(0[1-9]|1[0-2])$|'
                  r'^(19\d\d|20\d\d)$')
    
    numeric_regex = r'^[-+]?\d*\.?\d+$'
    
    regexs = {
        'CATCH_LOT': (r'^C\d{3}[A-Z]$', 
                      'like C123A'),
        'DATE_OF_COLLECTION': (date_regex, 'in YYYY-MM-DD format'),
        'DECIMAL_LATITUDE': (r'^[-+]?([0-8]\d|\d)(\.\d+)?$', 
                             'between -90 and 90'),
        'DECIMAL_LONGITUDE': (r'^[-+]?(1[0-7]\d|\d\d|\d)(\.\d+)?$', 
                              'between -180 and 180'),
        'WHAT_3_WORDS': (r'^///[a-z]+\.[a-z]+\.[a-z]+$', 
                         'like ///one.two.three'),
        'TIME_OF_COLLECTION': (r'^(?:[01]\d|2[0-3]):[0-5]\d$|^(?:[01]\d|2[0-3]):[0-5]\d:[0-5]\d$', 
                              'in HH:MM format'),
        'DURATION_OF_COLLECTION': (r'^P(?:\d+Y)?(?:\d+M)?(?:\d+D)?(?:T(?:\d+H)?(?:\d+M)?(?:\d+S)?)?$', 
                                   'in P[n]Y[n]M[n]DT[n]H[n]M[n]S format'),
        'DATE_OF_PLATING': (date_regex, 'in YYYY-MM-DD format'),
        'DATE_OF_PRESERVATION': (date_regex, 'in YYYY-MM-DD format'),
        'ELEVATION': (numeric_regex, 'only a number (in metres)'),
        'DNA_EXTRACT_VOLUME_PROVIDED': (numeric_regex, 'a number (in microlitres)'),
        'DNA_EXTRACT_CONCENTRATION': (numeric_regex, 'a number (in nanograms per microlitre)')
    }
    
    is_valid_regex = series.str.match(regexs[col][0])
    if not is_valid_regex.all():
        offending_values = list(series[~is_valid_regex].unique())
        s = index_ranges(series[~is_valid_regex])
        msg = (f'{col} format incorrect for SERIES {s}: found {offending_values} - '
              f'expected to be {regexs[col][1]}')
        if col == 'CATCH_LOT':
            logging.warning(msg)
        else:
            logging.error(msg)
        

In [ ]:
def validate_plates_wells(df, contrib_df, plate_col='RACK_OR_PLATE_ID', well_col='TUBE_OR_WELL_ID', bioscan=False):
    
    bioscan_partners_fn = '../data/bioscan_partners.tsv'
    
    # expect only complete 96-well plates
    logging.debug(f'validating {plate_col} and {well_col} columns')
    
    empty_rows = (df[plate_col] == '') | (df[well_col] == '')
    if empty_rows.any():
        logging.info(f'found and excluded {empty_rows.sum()} empty rows based on {plate_col} and {well_col}')
        df = df.loc[~empty_rows]
        
    # plate names validation
    plates = df[plate_col].drop_duplicates()
    plate_name_template = ('^[A-Z]{4}-[0-9]{3}$|^[A-Z]{4}_[0-9]{3}$' if bioscan else '^[A-Z]{4}_[0-9]{3}$')
    wrong_plate_names = plates[~plates.str.match(plate_name_template, na='')]
    if len(wrong_plate_names) > 0:
        logging.error(f'plate names {wrong_plate_names.to_list()} do not match template ABCD_1234')
    # check plate name prefixes against partner codes
    # and assign GAL
    plate_prefixes = plates.str.slice(0,4)
    if bioscan:
        partners_df = pd.read_csv(bioscan_partners_fn, sep='\t', dtype=str)
        assert partners_df['partner_code'].is_unique, f'duplicated partner codes found in {bioscan_partners_fn}'
        unknown_prefixes = (~plate_prefixes.isin(partners_df['partner_code']))
        
        possible_partner_codes = plate_prefixes.value_counts().index.to_list()
        if len(possible_partner_codes) > 1:
            logging.error(f'only one plate ID prefix expected, found multiple: {possible_partner_codes}')
        selected_partner_code = possible_partner_codes[0]
        selected_partner_df = partners_df.query(f'partner_code == "{selected_partner_code}"')
        if selected_partner_df.shape[0] == 0:
            logging.error(f'partner code {selected_partner_code} not found in {bioscan_partners_fn}, '
                          f'using "Sanger Institute" as default partner')
            gal = "Sanger Institute"
        else:
            gal = selected_partner_df['gal'].iloc[0]
        logging.info(f'selected GAL "{gal}" for partner code "{selected_partner_code}"')
    else:
        # anospp
        unknown_prefixes = (~plate_prefixes.isin(contrib_df['PARTNER_CODE']))
        gal = "Sanger Institute"
    if unknown_prefixes.any():
        logging.error(f'plate ID prefixes not recognised for {plates[unknown_prefixes].to_list()}')
        
    # add 96-well plate well IDs to validation
    row_id = list('ABCDEFGH')
    col_id = range(1,13)
    expected_wells = [r + str(c) for (r,c) in itertools.product(row_id, col_id)]
        
    for plate, pdf in df.groupby(plate_col):
        
        if bioscan and (pdf[well_col] == 'PLATE_ONLY').any():
            logging.debug(f'skipping well validation for PLATE_ONLY plate {plate}')
            continue
        
        # check for well duplicates
        dup_wells =  pdf[well_col].duplicated()
        if dup_wells.any():
            logging.error(f'duplicate {well_col} for plate {plate}: {pdf.loc[dup_wells, well_col].unique()}')
        # check for non A1...H12 wells
        observed_wells_set = set(pdf[well_col])
        expected_wells_set = set(expected_wells)
        if observed_wells_set != expected_wells_set:
            msg = f'in {well_col} for plate {plate}, '
            if len(expected_wells_set - observed_wells_set) > 0:
                msg += f'wells {expected_wells_set - observed_wells_set} are missing'
            if len(observed_wells_set - expected_wells_set) > 0:
                if msg.endswith('missing'):
                    msg += ', '
                msg += f'wells {observed_wells_set - expected_wells_set} are excessive'
            logging.error(msg)
    
    logging.info(f'{df.shape[0]} samples found across {df[plate_col].nunique()} plates')
    

    return df, gal
        
# df = validate_plates_wells(df, contrib_df, 'RACK_OR_PLATE_ID', 'TUBE_OR_WELL_ID')

In [ ]:
def expand_plate_only(df, plate_col='RACK_OR_PLATE_ID', well_col='TUBE_OR_WELL_ID'):
    
    # add 96-well plate well IDs for expansion
    row_id = list('ABCDEFGH')
    col_id = range(1,13)
    expected_wells = [r + str(c) for (c,r) in itertools.product(col_id, row_id)]
    
    blank_wells = {
        'PLATE_ONLY':['H12'], # v2
        'PLATE_ONLY_1_BLANK':['H12'], # v3
        'PLATE_ONLY_2_BLANKS':['G12','H12'], # v3
    }
    
    if df[well_col].isin(blank_wells.keys()).any():
        pdfs = []

        for plate, pdf in df.groupby(plate_col):
            # If plate level only metadata is being entered
            # put “PLATE_ONLY” in well 
            # and use only one row to capture the metadata for the whole plate
            if pdf[well_col].isin(blank_wells.keys()).any():
                
                if pdf.shape[0] > 1:
                    logging.error(f'expected one row in PLATE_ONLY plate {plate}, found {pdf.shape[0]} - '
                                  f'extracting metadata from first row only')
                plate_only_val = pdf[well_col].iloc[0]
                logging.info(f'found {plate_only_val} plate {plate}, expanding to 96 well rows, '
                             f'keeping {", ".join(blank_wells[plate_only_val])} blank')
                # expand to 96 rows
                pdf = pd.DataFrame(pdf.iloc[0] for i in range(len(expected_wells)))
                pdf[well_col] = expected_wells
                
                for blank_well in blank_wells[plate_only_val]:
                    for col in pdf.columns:
                        if col == 'ORGANISM_PART':
                            pdf.loc[pdf[well_col] == blank_well, col] = 'NOT_APPLICABLE'
                        elif col not in ['SERIES','CATCH_LOT','RACK_OR_PLATE_ID',
                                         'TUBE_OR_WELL_ID','ORGANISM_PART','PRESERVATIVE_SOLUTION']:
                            pdf.loc[pdf[well_col] == blank_well, col] = ''
            pdfs.append(pdf)
            
        df = pd.concat(pdfs).reset_index(drop=True)
        df.index += 1
        df.index.name = 'SERIES'
        
    else:
        logging.debug('no PLATE_ONLY rows found')
    
    return df
    
        

In [ ]:
def check_blanks(df):
    
    logging.debug('checking and excluding blank samples based on ORGANISM_PART being NOT_APPLICABLE')    
    
    # blank criterion
    is_blank = (df['ORGANISM_PART'] == 'NOT_APPLICABLE')
    
    blank_df = df[is_blank]
    
    non_blank_df = df[~is_blank]
    
    # last well of plate expected to be blank
    non_blank_last_well_df = non_blank_df.query('TUBE_OR_WELL_ID == "H12"')
    if non_blank_last_well_df.shape[0] > 0:
        logging.error(f'last well H12 is not blank at SERIES {index_ranges(non_blank_last_well_df)}: '
                      f'in ORGANISM_PART, expected "NOT_APPLICABLE", '
                      f'found {non_blank_last_well_df.ORGANISM_PART.to_list()}, '
                      f'these samples will be included in further analysis',
                        
        )
        
    for col in df.columns:
        if col not in ['SERIES','CATCH_LOT','RACK_OR_PLATE_ID',
                       'TUBE_OR_WELL_ID','ORGANISM_PART','PRESERVATIVE_SOLUTION',
                       'OTHER_INFORMATION','MISC_METADATA']:
            excessive_blank_info_df = blank_df[blank_df[col] != '']
            if excessive_blank_info_df.shape[0] > 0:
                logging.error(f'{col} column values non-empty for blank samples '
                              f'at SERIES {index_ranges(excessive_blank_info_df)} - these should be deleted')
            
    
    logging.info(f'{is_blank.sum()} blanks located across {df.RACK_OR_PLATE_ID.nunique()} plates, '
                 f'{non_blank_df.shape[0]} samples of {df.shape[0]} left for downstream analysis')
    
    
    return is_blank

# print(df.shape)
# is_blank = check_blanks(df)
# print(df[~is_blank].shape)

In [ ]:
def validate_values(col, df, valid_dict, sep=None, na_values=[], level='e'):
    
    logging.debug(f'validating for allowed values in {col} column')
    
    if col not in df.columns:
        logging.error(f'{col} column not found in manifest')
        return
    if col not in valid_dict.keys():
        logging.error(f'{col} column not found in validation sheet')
        return
    assert level in ('i','w','e'), '{!r} invalid logging level for validate_values'.format(level)
    
    series = df[col]
    series = exclude_missing(series, na_values)
    
    col_values = set(series.unique())
    # use separator to split values
    if sep:
        sep_col_values = list()
        for v in col_values:
            sep_col_values.extend([x.strip() for x in v.split(sep)])
        col_values = set(sep_col_values)
    
    valid_values = set(valid_dict[col])
    
    invalid_values = col_values - valid_values
    
    if len(invalid_values) > 0:
        if sep:
            invalid_value_series = index_ranges(series[series.str.contains('|'.join(invalid_values), regex=True)])
        else:
            invalid_value_series = index_ranges(series[series.isin(invalid_values)])
        msg = f'invalid values in {col} column, SERIES {invalid_value_series}: {invalid_values}'
        if level == 'i':
            logging.info(msg)
        elif level == 'w':
            logging.warning(msg)
        elif level == 'e':
            logging.error(msg)
#     else:
#         logging.info('all values valid in {!r}'.format(col))
            
# validate_values('ORGANISM_PART', df, valid_dict, sep=" | ", na_values=[], level='e')

In [ ]:
def compare_dates_text(before_col, after_col, df):
    
    logging.debug(f'checking that {before_col} is earlier than {after_col}')
    
    for col in (before_col, after_col):
        if col not in df.columns:
            logging.error(f'{col} column not found in manifest')
            return
    # invalid date formats or empty string converted to NaT
    before_series = pd.to_datetime(df[before_col], format='%Y-%m-%d', errors='coerce').copy()
    after_series = pd.to_datetime(df[after_col], format='%Y-%m-%d', errors='coerce').copy()
    
    date_conflict = before_series > after_series
    
    s = index_ranges(df[date_conflict])
    
    if date_conflict.any():
        logging.error(f'{before_col} column values are later than {after_col} column values in SERIES {s}')

In [ ]:
def validate_country_and_coordinates(df, fn, na_values=[], bioscan=False):
    
    logging.debug('validating COUNTRY_OF_COLLECTION against DECIMAL_LATITUDE and DECIMAL_LONGITUDE')
    
    country_col, lat_col, lon_col = 'COUNTRY_OF_COLLECTION', 'DECIMAL_LATITUDE', 'DECIMAL_LONGITUDE'

    try:
        loc_df_complete = df[[country_col, lat_col, lon_col]].copy()
    except:
        logging.error(f'some of {country_col} {lat_col} {lon_col} columns not found in manifest')
        return
    loc_df_isna = (loc_df_complete.isin(na_values)).all(axis=1)
    if loc_df_isna.any():
        logging.info(f'removing {loc_df_isna.sum()} missing data ({na_values}) samples from coordinate analysis')
    loc_df_complete = loc_df_complete[~loc_df_isna].copy()
    
    # coordinates in geopy format
    loc_df_complete['coord'] = loc_df_complete.apply(lambda x: '{}, {}'.format(
            x[lat_col], x[lon_col]), axis=1)
    
    # get location data for coordinates
    # use local copy of web query results for re-runs
    # this 
    loc_fn = fn+'_loc.pkl'
    if os.path.isfile(loc_fn):
        locations = pickle.load(open(loc_fn, "rb"))
    else:
        # web map server - openstreetmaps
        logging.debug('querying coordinates')
        locator = Nominatim(user_agent='myGeocoder')
        rgeocode = RateLimiter(locator.reverse, min_delay_seconds=1)

        locations = dict()
        for c in loc_df_complete.coord.unique():
            # pre-fill with unknown country
            locations[c] = {'address':{'country':'UNKNOWN'}}
            # check coordniate correctness
            try:
                lat, lon = c.split(', ')
                lat, lon = float(lat), float(lon)
            except:
                unparsed_df = df[(df[lat_col] == str(lat)) & df[lon_col] == str(lon)]
                logging.error(
                    f'problem parsing coordinates {c} at SERIES {index_ranges(unparsed_df)}'
                )
                continue
            if abs(lat) > 90:
                logging.error(
                    f'invalid DECIMAL_LATITUDE {lat} at SERIES {index_ranges(df[df[lat_col] == str(lat)])}'
                    f', should be in [-90,90]')
                continue
            if abs(lon) > 180:
                logging.error(
                    f'invalid DECIMAL_LONGITUDE {lon} at SERIES {index_ranges(df[df[lon_col] == str(lon)])}'
                    f', should be in [-180,180]')
                continue
            # web query
            location = rgeocode(c, language='en-gb')
            # rgeocode returns empty location outside of counries and in some other situations
            if location is not None:
                locations[c] = location.raw

        # save locations to file
        pickle.dump(locations, open(loc_fn, "wb"))
        
    # parse country from partner input
    loc_df_complete['partner_country'] = loc_df_complete[country_col].str.strip().str.upper()
    
    # extract countries from location data
    loc_countries = dict()
    for coord in locations.keys():
        
        lat, lon = coord.split(', ')
        coord_series = index_ranges(df.query(f'({lat_col} == "{lat}") & ({lon_col} == "{lon}")'))
                    
        coord_country = locations[coord]['address']['country'].upper()
        loc_countries[coord] = coord_country
        
        partner_countries = loc_df_complete.loc[loc_df_complete.coord == coord, 'partner_country']
        if partner_countries.nunique() > 1:
            logging.error(
                f'multiple COUNTRY_OF_COLLECTION records found for coordinates {coord}, SERIES {coord_series}: '
                f'{partner_countries.unique()}, skipping coordinate validation'
            )
            continue
        if partner_countries.shape[0] == 0:
            logging.error(f'no COUNTRY_OF_COLLECTION records found for coordinates {coord}, SERIES {coord_series}')
            continue
        partner_country = partner_countries.iloc[0]
        if coord_country == 'UNKNOWN':
            logging.warning(f'could not locate country for coordinates {coord}, '
                            f'COUNTRY_OF_COLLECTION {partner_country}, SERIES {coord_series}')
        elif partner_country != coord_country:
            logging.error(f'country mismatch for coordinates {coord}, SERIES {coord_series}: '
                          f'COUNTRY_OF_COLLECTION indicated as {partner_country}, '
                          f'while coordinates point to {coord_country}')
    
    # countries based on coordinates
    loc_df_complete['coord_country'] = loc_df_complete['coord'].replace(loc_countries)
    country_mismatch = (loc_df_complete.coord_country != loc_df_complete.partner_country)

#     if country_mismatch.any():
#         logging.error('coordinates do not match country for SERIES: {}'.format(
#                 country_mismatch[country_mismatch].index.to_list()))
    
    # location data can be re-used, e.g. as an additional field
    return loc_df_complete
# df.loc[2,'DECIMAL_LATITUDE'] = '65'
# loc_test = validate_country_and_coordinates(df, anospp_fn)
# loc_test

In [ ]:
def validate_taxonomy(df, ncbi, na_values = [], anospp=False, add_taxids=False):

    logging.debug('validating taxonomy against NCBI')
    
    if anospp:
        df['PREDICTED_ORDER_OR_GROUP'] = 'Diptera'
        df['PREDICTED_FAMILY'] = 'Culicidae'
        df['PREDICTED_GENUS'] = 'Anopheles'
        
        harbach_spp = []
        with open('../data/harbach_spp_201910.txt') as f:
            for line in f:
                harbach_spp.append('Anopheles ' + line.strip())
        harbach_spp = set(harbach_spp)
        
    tax_levels = {
        'PREDICTED_ORDER_OR_GROUP':'order',
        'PREDICTED_FAMILY':'family',
        'PREDICTED_GENUS':'genus',
        'PREDICTED_SCIENTIFIC_NAME':'species'
    }
    
    expected_ranks = {
            'order':('class','subclass','order'),
            'family':('suborder','superfamily','family','subfamily','tribe','subtribe'),
            'genus':('genus','subgenus'),
            'species':('species')
        }
    
    hierarchies = df[tax_levels.keys()].drop_duplicates().copy()
    
    hierarchies.columns = list(tax_levels.values())
        
    tax_info = dict()
    
    for tax_col, tax_level in tax_levels.items():
        
        logging.debug(f'validating {tax_col} column against NCBI')
        
        if tax_col not in df.columns:
                logging.error(f'{tax_col} column not found in manifest')
                continue
            
        tax_names = list(hierarchies[tax_level].unique())
        
        for na_value in na_values:
            try:
                tax_names.remove(na_value)
            except:
                pass 
            
        for i, tax_name in enumerate(tax_names):
            if len(tax_name) == 0:
                continue
            corr_tax_name = tax_name[0].upper() + tax_name[1:].lower()
            if corr_tax_name != tax_name and tax_name != 'blank sample':
                s = index_ranges(df.query(f'{tax_col} == "{tax_name}"'))
                logging.info(f'{tax_col} column, SERIES {s}'
                             f': unexpected case for "{tax_name}", '
                             f'changing to "{corr_tax_name}" for validation and output')
            tax_names[i] = corr_tax_name
        
        tax_info[tax_level] = ncbi.get_name_translator(tax_names) 
        
        unmatched_names = set(tax_names) - set(tax_info[tax_level].keys())
        if len(unmatched_names) > 0:
            for tname in unmatched_names:
                s = index_ranges(df[df[tax_col].str.match(f'^{re.escape(tname)}$', case=False)])
                if tax_level == 'species' and anospp:
                    if tname in harbach_spp:
                        logging.warning(f'{tax_col} column, SERIES {s}:'
                                        f' "{tname}" found in Harbach list, but not in NCBI Taxonomy')
                    else:
                        logging.error(f'{tax_col} column, SERIES {s}'
                                      f': "{tname}" not found in both Harbach list and NCBI Taxonomy')
                else:
                    logging.error(f'{tax_col} column, SERIES {s}: "{tname}" not found in NCBI Taxonomy')
        
        
        
        for tname, tids in tax_info[tax_level].items():
            
            ranks = ncbi.get_rank(tids)
            
            upd_tid = tids[0]
            
            s = index_ranges(df[df[tax_col].str.match(f'^{re.escape(tname)}$', case=False)])
                        
            if len(tids) == 1:
                if ranks[upd_tid] not in expected_ranks[tax_level]: 
                    logging.error(f'{tax_col} column, SERIES {s}: found unexpected rank for "{tname}" '
                                      f'(taxid {upd_tid}): "{ranks[upd_tid]}"')
                    
            if len(tids) > 1:            
                for tid, rank in ranks.items():
                    if rank in expected_ranks[tax_level] and len(tids) > 1:
                        logging.debug(f'{tax_col} column, SERIES {s}: using only first matching rank '
                                      f'for "{tname}" (taxid {tid}): "{rank}"')
                        upd_tid = tid
                        break
                else:
                    logging.error(f'{tax_col} column, SERIES {s}: could not find matching rank '
                                  f'for "{tname}" - using taxid {upd_tid}: "{ranks[upd_tid]}"')
                    
            tax_info[tax_level][tname] = upd_tid
        
        #logging.info(f'{tax_level} {tax_info[tax_level]}')
                    
    # check consistency of taxonomy
    for _, r in hierarchies.iterrows():
        
        if r.order in na_values:
            continue
        try:
            order_id = tax_info['order'][r.order]
        except KeyError:
            logging.debug(f'PREDICTED_ORDER_OR_GROUP value "{r.order}" not in NCBI Taxonomy, '
                          f'skipping taxonomy consistency check')
            continue
            
        if r.genus in na_values:
            continue
        try:
            family_id = tax_info['family'][r.family]
            
            family_lineage = ncbi.get_lineage(family_id)
            
            s = index_ranges(df[df['PREDICTED_FAMILY'].str.match(f'^{re.escape(r.family)}$', case=False)])
            
            if order_id not in family_lineage:
                logging.error(f'PREDICTED_FAMILY column, SERIES {s}: family "{r.family}" (taxid {family_id}) '
                              f'does not belong to order "{r.order}" (taxid {order_id})')
        except KeyError:
            logging.debug(f'PREDICTED_FAMILY value "{r.family}" not in NCBI Taxonomy, '
                          f'skipping taxonomy consistency check')
            continue
            
        if r.genus in na_values:
            continue
        try:
            genus_id = tax_info['genus'][r.genus]
            
            genus_lineage = ncbi.get_lineage(genus_id)
            
            s = index_ranges(df[df['PREDICTED_GENUS'].str.match(f'^{re.escape(r.genus)}$', case=False)])
            
            if order_id not in genus_lineage:
                logging.error(
                    f'PREDICTED_GENUS column, SERIES {s}: '
                    f'genus "{r.genus}" (taxid {genus_id}) does not belong to "{r.order}" (taxid {order_id})')
            if family_id not in genus_lineage:
                logging.error(
                    f'PREDICTED_GENUS column, SERIES {s}: '
                    f'genus "{r.genus}" (taxid {genus_id}) does not belong to "{r.family}" (taxid {family_id})')
        except KeyError:
            logging.debug(f'PREDICTED_GENUS value "{r.genus}" not in NCBI Taxonomy, '
                          f'skipping taxonomy consistency check')
            continue
            
        if r.species in na_values:
            continue
        try:
            species_id = tax_info['species'][r.species]
            
            species_lineage = ncbi.get_lineage(species_id)
            
            s = index_ranges(df[df['PREDICTED_SCIENTIFIC_NAME'].str.match(f'^{re.escape(r.species)}$', case=False)])
            
            if order_id not in species_lineage:
                logging.error(
                    f'PREDICTED_SCIENTIFIC_NAME column, SERIES {s}: '
                    f'species "{r.species}" (taxid {species_id}) does not belong to "{r.order}" (taxid {order_id})')
            if family_id not in species_lineage:
                logging.error(
                    f'PREDICTED_SCIENTIFIC_NAME column, SERIES {s}: '
                    f'species "{r.species}" (taxid {species_id}) does not belong to "{r.family}" (taxid {family_id})')
            if genus_id not in species_lineage:
                logging.error(
                    f'PREDICTED_SCIENTIFIC_NAME column, SERIES {s}: '
                    f'species "{r.species}" (taxid {species_id}) does not belong to "{r.genus}" (taxid {genus_id})')
            elif r.species.split(' ')[0] != r.genus:
                logging.error(
                    f'PREDICTED_SCIENTIFIC_NAME column, SERIES {s}: '
                    f'species "{r.species}" (taxid {species_id}) states a different genus name than '
                    f'"{r.genus}" (taxid {genus_id})')
        except KeyError:
            logging.info(f'PREDICTED_SCIENTIFIC_NAME value "{r.species}" not in NCBI Taxonomy, '
                         f'skipping taxonomy consistency check')
            continue
    
    if add_taxids:
        for tc in tax_levels.keys():
            df[f'{tc}_TAXID'] = df[tc].replace(tax_info[tax_levels[tc]])
    
    # only fix taxonomy case after validation to get naming 
    for tax_col in tax_levels.keys():
        df[tax_col] = df[tax_col].str.capitalize()
            
    return df
        
                
# validate_taxonomy(df, ncbi, anospp=True)

In [ ]:
def validate_specimen_id_risk(df):
    
    logging.debug(f'validating SPECIMEN_IDENTITY_RISK column')
    
    if 'SPECIMEN_IDENTITY_RISK' not in df.columns:
        logging.error(f'SPECIMEN_IDENTITY_RISK column not found in manifest')
        return
    
    # missing species name, but no idenitity risk
    invalid_risk = ((df.PREDICTED_SCIENTIFIC_NAME == '') & (df.SPECIMEN_IDENTITY_RISK == 'N'))
    
    if invalid_risk.any():
        logging.error(f'SERIES {index_ranges(df.loc[invalid_risk])}: with no PREDICTED_SCIENTIFIC_NAME, '
                      f'SPECIMEN_IDENTITY_RISK values should be blank or "Y", not "N"')

# validate_specimen_id_risk(df)

In [ ]:
def validate_freetext(col, df, na_values=['']):
    
    logging.debug(f'validating freetext field characters in {col} column')
    
    if col not in df.columns:
        logging.error(f'{col} column not found in manifest')
        return
    series = df[col]
    series = exclude_missing(series, na_values)
    
    regex = '^[A-z0-9.,\-_ ]+$'
    
    is_valid_freetext = series.str.match(regex)
    if not is_valid_freetext.all():
        s = index_ranges(series.loc[~is_valid_freetext])
        logging.debug(f'{col} column, SERIES {s}: found non-standard characters - regex: "{regex}"')

        
# validate_freetext('IDENTIFIED_HOW', df)

In [ ]:
def add_sts_cols(df, contrib_df, gal, bioscan=True, v='v2'):
    
    logging.debug('adding STS columns to manifest')
    
    is_blank = (df['ORGANISM_PART'] == 'NOT_APPLICABLE')
    
    df['SPECIMEN_ID'] = df['RACK_OR_PLATE_ID'] + '_' + df['TUBE_OR_WELL_ID']
    dup_specimen_id = df['SPECIMEN_ID'].duplicated()
    if dup_specimen_id.any():
        logging.error(f'duplicate SPECIMEN_ID generated: {df.SPECIMEN_ID[dup_specimen_id].unique()}')
    df['SCIENTIFIC_NAME'] = 'unidentified'
    df.loc[is_blank, 'SCIENTIFIC_NAME'] = 'blank sample'
    df['TAXON_ID'] = '32644'
    df.loc[is_blank, 'TAXON_ID'] = '2582415'
    df['GAL'] = gal
    df['SYMBIONT'] = 'TARGET'
    df['REGULATORY_COMPLIANCE'] = 'Y'
    df['HAZARD_GROUP'] = 'HG1'
    if bioscan and v == 'v2':
        logging.info('auto-filling CATCH_SOLUTION and AMOUNT_OF_CATCH_PLATED columns for bioscan manifest v2')
        df['CATCH_SOLUTION'] = '100%_ETHANOL'
        df['AMOUNT_OF_CATCH_PLATED'] = 'ALL_SPECIMENS_PLATED'
        logging.info('dropping IDENTIFIER_AFFILIATION column for bioscan manifest v2')
        if (df['IDENTIFIER_AFFILIATION'] != '').any():
            logging.warning('IDENTIFIER_AFFILIATION has some data filled in - '
                            'note this column will be removed from output')
        logging.info('dropping IDENTIFIER_AFFILIATION column for bioscan manifest v2')
        df = df.drop(columns=['IDENTIFIER_AFFILIATION'])
    # add contributors - delimiters checked in validate_contributors
    contrib_series = contrib_df['FULL_NAME'] + ';' + \
        contrib_df['PRIMARY_AFFILIATION'] + ';' + \
        contrib_df['EMAIL ADDRESS'] + ';' + \
        contrib_df['CONTRIBUTION']
    df['CONTRIBUTORS'] = '|'.join(list(contrib_series))
    
    logging.info('dropping MISC_METADATA column')
    if (df['MISC_METADATA'] != '').any():
        logging.warning('MISC_METADATA has some data filled in - note this column will be removed from output')
    df = df.drop(columns=['MISC_METADATA'])
    
    return df

# add_sts_cols(df, contrib_df, gal='Sanger Institute');

In [ ]:
def write_sts_manifest(df, input_fn, validation_version):
    
    output_fn = input_fn.rstrip('.xlsx') + '_' + validation_version + '_for_sts.xlsx'
        
    logging.info(f'writing STS manifest to "{output_fn}"')
    
    df.to_excel(output_fn, sheet_name='Metadata Entry')
    